In [18]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV 
from sklearn.grid_search import RandomizedSearchCV 

from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer
import pymorphy2
import re

import sys

reload(sys)
sys.setdefaultencoding('utf8')

In [19]:
import re
somestring='Стояли звери, **d asdd () 1) - 3123 около двери tets test'
#re.compile('[^a-zA-ZА-Яа-яёЁ]')
rx = re.compile('[^a-zA-ZА-Яа-яёЁ0-9]')
res = rx.sub(' ', somestring).strip()
res

'\xd0\xa1\xd1 \xd0\xbe\xd1\x8f\xd0\xbb\xd0\xb8 \xd0\xb7\xd0\xb2\xd0\xb5\xd1 \xd0\xb8    d asdd    1    3123 \xd0\xbe\xd0\xba\xd0\xbe\xd0\xbb\xd0\xbe \xd0\xb4\xd0\xb2\xd0\xb5\xd1 \xd0\xb8 tets test'

In [92]:
morph = pymorphy2.MorphAnalyzer()
morph.parse(u'камерой,')[0].normal_form

u'\u043a\u0430\u043c\u0435\u0440\u043e\u0439,'

In [21]:
stemmer = RussianStemmer()
stemmer.stem(u'камерой,')

u'\u043a\u0430\u043c\u0435\u0440\u043e\u0439,'

In [22]:
data = pd.read_csv('./data/data.csv')
del data['Unnamed: 0']

In [23]:
data.head()

,grade,link,text,content
0,5,https://technopoint.ru/product/42db0e393aab333...,"большой экран, мощный аккумулятор, режим для ч...",https://technopoint.ru/product/42db0e393aab333...
1,5,https://technopoint.ru/product/da39313444f8333...,Телефоном пользуюсь немного и это скорее перв...,https://technopoint.ru/product/da39313444f8333...
2,4,https://technopoint.ru/product/44b2f37e4035333...,"Цена, брал за 7900 руб. Яркий экран, хорошо ...",https://technopoint.ru/product/44b2f37e4035333...
3,5,https://technopoint.ru/product/305c1b434efe333...,Цельный качественный корпус с большой батарее...,https://technopoint.ru/product/305c1b434efe333...
4,5,https://technopoint.ru/product/0ecb5f413466333...,"камера, металл корпус , андроид отличный бюдже...",https://technopoint.ru/product/0ecb5f413466333...


In [24]:
data.grade.value_counts()

5    8766
4    2408
3    1060
1     652
2     640
Name: grade, dtype: int64

In [30]:
data['label'] = data.grade.apply(lambda x: 0 if x < 4 else 1)

In [95]:
data.label.value_counts()

1    11174
0     2352
Name: label, dtype: int64

In [116]:
corpus = data[['text', 'label']]

def clean_str(string):
    morph = pymorphy2.MorphAnalyzer()
    
    symbols = [
        ',', '.', '-', '*', '#', ')', '(', '/', '<', '>', ':', '+', '?', '!', '"', '"', '%', '=', '\\', '}'
    ]
    
    for symbol in symbols:
        print string
        string = str(string).replace(symbol, ' ')
        
    words = string.split()
    normalized_words = []
    
    for word in words:
        normalized_words.append(morph.parse(unicode(word.strip()))[0].normal_form)
        
    string = unicode(' '.join(normalized_words))
    
    return string

#corpus['text'] = corpus.text.apply(clean_str)

In [117]:
drop_indices = np.random.choice(corpus[corpus.label == 1].index, 11000, replace=False)
corpus = corpus.drop(drop_indices)
corpus.label.value_counts()

0    2352
1     174
Name: label, dtype: int64

In [118]:
def clean_str(string):
    morph = pymorphy2.MorphAnalyzer()
    
    symbols = [
        ',', '.', '-', '*', '#', ')', '(', '/', '<', '>', ':', '+', '?', '!', '"', '"', '%', '=', '\\', '}'
    ]
    
    for symbol in symbols:
        string = str(string).replace(symbol, ' ')
        
    words = string.split()
    normalized_words = []
    
    for word in words:
        normalized_words.append(morph.parse(unicode(word.strip()))[0].normal_form)
        
    string = unicode(' '.join(normalized_words))
    
    return string

corpus['text'] = corpus.text.apply(clean_str)

In [119]:
corpus.head()

,text,label
6,пользоваться телефон 4 месяц не однократно лов...,0
9,отвратительный оболочка от samsung на android ...,0
20,1 очень медленный автоматический подстройка яр...,0
21,тупить пользоваться имя почти половина год и з...,0
24,не работать сервис google о это честно написат...,0


In [120]:
corpus.text.values[0]

u'\u043f\u043e\u043b\u044c\u0437\u043e\u0432\u0430\u0442\u044c\u0441\u044f \u0442\u0435\u043b\u0435\u0444\u043e\u043d 4 \u043c\u0435\u0441\u044f\u0446 \u043d\u0435 \u043e\u0434\u043d\u043e\u043a\u0440\u0430\u0442\u043d\u043e \u043b\u043e\u0432\u0438\u0442\u044c \u0434\u0438\u043a\u0438\u0439 \u0437\u0430\u0432\u0438\u0441\u043e\u043d \u043f\u0440\u0438\u0445\u043e\u0434\u0438\u0442\u044c\u0441\u044f \u0441\u043d\u0438\u043c\u0430\u0442\u044c \u0430\u043a\u0443\u043c\u0430 \u043a\u0430\u043c\u0435\u0440\u0430 \u043e\u0442\u043a\u0430\u0437\u0430\u0442\u044c \u0440\u0430\u0431\u043e\u0442\u0430\u0442\u044c \u043d\u0430\u043f\u0440\u043e\u0447\u044c \u043e\u0441\u0442\u0430\u0442\u044c\u0441\u044f \u0442\u043e\u043b\u044c\u043a\u043e \u0444\u0440\u043e\u043d\u0442\u0430\u043b\u043a\u0430 \u043e\u043d\u0430 \u043d\u0430\u0444\u0438\u0433 \u044f \u043d\u0435 \u043d\u0443\u0436\u043d\u044b\u0439 \u0432\u043c\u0435\u0441\u0442\u0435 \u0441 \u043a\u0430\u043c\u0435\u0440\u0430 \u0441\u0434\u04

In [30]:
cv = StratifiedKFold(corpus.label, n_folds=10, shuffle=True, random_state=1)

pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', LogisticRegression())
])

pipeline_params = {'classifier__penalty': ['l1', 'l2']}

grid = GridSearchCV(pipeline, pipeline_params, cv=cv, refit=True, verbose=1, n_jobs=4)

grid.fit(corpus.text.values.astype('U'), corpus.label)
best = grid.best_estimator_
print(
    "Accuracy (TfidfVectorizer + LogisticRegression): {}, params {}" . format(grid.best_score_, grid.best_params_)
)
grid.best_score_

0.980777761348514

In [31]:
grid.best_params_

{'classifier__penalty': 'l2'}

In [28]:
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2), max_df=1.0, analyzer='word', use_idf=True,
    norm='l2', stop_words=stopwords.words('russian')
)
X_vect = vectorizer.fit_transform(corpus.text.values.astype('U'))

classifier = LogisticRegression(penalty='l2')
fitted = classifier.fit(X_vect, corpus.label)

In [29]:
X_test = vectorizer.transform([unicode('Пользуюсь 3 месяца.  На выбор повлиял большой 5,5 дюймовый  HD экран и мощный аккумулятор 4500 м Ач.  Экран действительно весьма большой и с качественной картинкой.  Удобно читать книги на таком экране.  Тем более что есть режим для комфортного чтения, свет на экране становится более теплым для глаз.  Можно к телефону подключать клавиатуру, компьютер и флэшку через  USB кабель.  Это мне часто помогает в работе.  Дома компа временно нет- отдал в ремонт,  а отчеты нужно отправлять вовремя, поэтому приходится пользоваться телефоном дома.  Аккумулятора хватает на 2 дня, заряжаю через день.  Камера снимает в разных режимах.  Я снимаю обычно серийную съемку, камера делает сразу 30 снимков, из них потом легче выбирать.')])
fitted.predict_proba(X_test)

array([[ 0.23646965,  0.76353035]])

In [30]:
fitted.predict(X_test)

array([1])